In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from state_cleaner import *
import datetime
import csv
from scipy import stats as st
import itertools
import re 

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'C:\\Users\\Noah\\CovidMobile'

In [43]:
df = pd.read_csv('data/compiled_2020-04-26.csv')
df.shape

(155117, 106)

In [44]:
df.columns.values

array(['country_region_code', 'country_region', 'state_x', 'county_x',
       'date', 'retail', 'food_drugs', 'park', 'transit', 'work', 'homes',
       'source', 'state_abb_x', 'FIPS', 'geo', 'CENSUS_2010_POP',
       'POP_ESTIMATE_2012', 'POP_ESTIMATE_2016',
       'Less than a high school diploma, 2014-18',
       'High school diploma only, 2014-18',
       "Some college or associate's degree, 2014-18",
       "Bachelor's degree or higher, 2014-18",
       'Percent of adults with less than a high school diploma, 2014-18',
       'Percent of adults with a high school diploma only, 2014-18',
       "Percent of adults completing some college or associate's degree, 2014-18",
       "Percent of adults with a bachelor's degree or higher, 2014-18",
       'POP_ESTIMATE_2018', 'N_POP_CHG_2018', 'Births_2018',
       'Deaths_2018', 'NATURAL_INC_2018', 'INTERNATIONAL_MIG_2018',
       'DOMESTIC_MIG_2018', 'NET_MIG_2018', 'RESIDUAL_2018',
       'GQ_ESTIMATES_2018', 'R_birth_2018', 'R_death_20

In [45]:
dct = df[['state_abb_x','FIPS']].drop_duplicates().dropna()
dct.columns = ['neighbors_state','neighbors']
dct

,neighbors_state,neighbors
114,AL,1000.0
171,AL,1001.0
228,AL,1003.0
285,AL,1005.0
342,AL,1007.0
...,...,...
154856,WY,56037.0
154913,WY,56039.0
154970,WY,56041.0
155027,WY,56043.0


In [47]:
neighbors = df[['state_abb_x','FIPS']+list(df.filter(like='neighbors').columns.values)].drop_duplicates().dropna(subset=['state_abb_x','FIPS'])
neighbors

,state_abb_x,FIPS,neighbors1,neighbors2,neighbors3,neighbors4,neighbors5,neighbors6,neighbors7,neighbors8,neighbors9,neighbors10,neighbors11,neighbors12,neighbors13,neighbors14,neighbors15
114,AL,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,AL,1001.0,1001.0,1021.0,1047.0,1051.0,1085.0,1101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,AL,1003.0,1003.0,1025.0,1053.0,1097.0,1099.0,1129.0,12033.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,AL,1005.0,1005.0,1011.0,1045.0,1067.0,1109.0,1113.0,13061.0,13239.0,13259.0,NaN,NaN,NaN,NaN,NaN,NaN
342,AL,1007.0,1007.0,1021.0,1065.0,1073.0,1105.0,1117.0,1125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154856,WY,56037.0,8081.0,49009.0,49043.0,56007.0,56013.0,56023.0,56035.0,56037.0,56041.0,NaN,NaN,NaN,NaN,NaN,NaN
154913,WY,56039.0,16019.0,16043.0,16081.0,30031.0,56013.0,56023.0,56029.0,56035.0,56039.0,NaN,NaN,NaN,NaN,NaN,NaN
154970,WY,56041.0,49033.0,49043.0,56023.0,56037.0,56041.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155027,WY,56043.0,56003.0,56013.0,56017.0,56019.0,56025.0,56029.0,56043.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
for x in range(1,16,1):
    dct.columns = ['neighbors_state{}'.format(x),'neighbors{}'.format(x)]
    neighbors = neighbors.merge(dct,on=['neighbors{}'.format(x)],how='left')

In [58]:
neighbors['border_states'] = neighbors.filter(like='neighbors_state').T.agg([pd.unique]).T
neighbors['count_border_states'] = neighbors['border_states'].apply(len)-2

In [60]:
neighbors[neighbors['count_border_states']==3]

,state_abb_x,FIPS,neighbors1,neighbors2,neighbors3,neighbors4,neighbors5,neighbors6,neighbors7,neighbors8,...,neighbors_state8,neighbors_state9,neighbors_state10,neighbors_state11,neighbors_state12,neighbors_state13,neighbors_state14,neighbors_state15,border_states,count_border_states
82,AZ,4001.0,4001.0,4009.0,4011.0,4017.0,8083.0,35003.0,35006.0,35031.0,...,NM,NM,UT,NaN,NaN,NaN,NaN,NaN,"[AZ, CO, nan, NM, UT]",3
90,AZ,4015.0,4005.0,4012.0,4015.0,4025.0,6071.0,32003.0,32017.0,49025.0,...,UT,UT,NaN,NaN,NaN,NaN,NaN,NaN,"[AZ, CA, NV, nan, UT]",3
260,CO,8083.0,4001.0,8033.0,8067.0,8083.0,8111.0,35045.0,49037.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[AZ, nan, CO, NM, UT]",3
291,DE,10003.0,10001.0,10003.0,24015.0,24029.0,34015.0,34033.0,42029.0,42045.0,...,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[DE, MD, NJ, PA, nan]",3
1126,MD,24043.0,24001.0,24021.0,24043.0,42055.0,42057.0,51107.0,54003.0,54037.0,...,WV,WV,NaN,NaN,NaN,NaN,NaN,NaN,"[MD, PA, VA, WV, nan]",3
1131,MA,25003.0,9005.0,25003.0,25011.0,25013.0,25015.0,36021.0,36027.0,36083.0,...,NY,VT,NaN,NaN,NaN,NaN,NaN,NaN,"[CT, MA, NY, VT, nan]",3
1143,MA,25027.0,9013.0,9015.0,25011.0,25013.0,25015.0,25017.0,25021.0,25027.0,...,MA,NH,NH,RI,NaN,NaN,NaN,NaN,"[CT, MA, NH, RI, nan]",3
1660,NM,35045.0,4001.0,8007.0,8067.0,8083.0,35031.0,35039.0,35043.0,35045.0,...,NM,UT,NaN,NaN,NaN,NaN,NaN,NaN,"[AZ, CO, NM, UT, nan]",3
2533,UT,49037.0,4001.0,4005.0,4017.0,8033.0,8077.0,8083.0,8085.0,8113.0,...,CO,NM,UT,UT,UT,UT,UT,NaN,"[AZ, nan, CO, NM, UT]",3


In [61]:
df = df.merge(neighbors[['state_abb_x','FIPS','count_border_states']],on=['state_abb_x','FIPS'])

In [65]:
df[(df['count_border_states']>0)&(df['count_border_states'].notnull())]['state_x'].value_counts()

Georgia           2335
Kentucky          2296
North Carolina    2271
Virginia          2218
Tennessee         2206
Missouri          2170
Texas             1985
Indiana           1973
Illinois          1930
Iowa              1783
Mississippi       1695
Wisconsin         1666
Pennsylvania      1653
Ohio              1533
Arkansas          1502
New York          1482
Alabama           1470
Minnesota         1451
West Virginia     1431
Louisiana         1225
Michigan          1194
Oklahoma          1165
Idaho             1159
South Carolina    1112
Maryland          1083
Kansas            1052
Oregon             936
South Dakota       929
New Jersey         912
Nebraska           880
Colorado           842
New Mexico         831
Florida            798
Wyoming            678
Montana            671
California         667
Washington         626
Utah               604
Nevada             525
Vermont            515
Massachusetts      456
Connecticut        456
New Hampshire      456
Arizona    